# Algoritmo de desarrollo orientado a objetos y numpy
Se busca probar las funciones y scripts para obtener una mayor rapidez de ejecucion.
Se generan los algoritmos necesarios para el desarrollo del algoritmo que compare ambas estructuras
Basandose en la estructura como esta en start_oo.py

In [1]:
# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
# herramientas para leer pdbs
import read_pdb_tools as rpt
# calculo de distancia
from scipy.spatial.distance import pdist, squareform
# libreria de tablas
import pandas as pd
# funciones de click generadas en pandas
import funciones_CLICK as fc
# iteradores
import itertools as it
# cuenta tiempo de ejecucion
import datetime
#distancia
from scipy.spatial import distance
#multiprocesamiento
import multiprocessing
from functools import partial

In [2]:
tiempo =  datetime.datetime.now()

In [3]:
# assert( len(sys.argv) > 1)
# lectura de archivo
file1 = 'pdbs/1xxa.pdb' # sys.argv[1]
file2 = 'pdbs/1tig.pdb' # sys.argv[2]

# numero de cliques, preguntar en el software para generalizarlo...
num_cliques = 3

# se define la estructura
pdb1 = rpt.PdbStruct("first")
pdb2 = rpt.PdbStruct("second")

# se lee el pdb y se agrega al objeto
pdb1.AddPdbData("%s" % file1)
pdb2.AddPdbData("%s" % file2)

# se obtienen los residuos que perteneces a la cadena de interes por default chain = 'A'
pdb11 = pdb1.GetResChain()
pdb22 = pdb2.GetResChain()
# se pega a la tabla la estructura secundaria
ss1 = pdb1.Get_SS(file1)
ss2 = pdb1.Get_SS(file2)

In [4]:
# se crea atributo a cada residuo de la estructura secundaria
for i, j in zip(pdb11, ss1.structure.values):
    setattr(i, 'structure', j)
for i, j in zip(pdb22, ss2.structure.values):
    setattr(i, 'structure', j)

In [5]:
def get_df_distancias(ref):
    """Funcion para obtener el dataframe de distancias de cada proteina"""
    # se generan listas con coordenadas y numero de atomo
    coord = [res.GetAtom('CA').coord for res in ref]
    index = [res.resi for res in ref]

    # calcula distancia y regresa dataframe
    distancias = []
    # se calcula la distancia euclidiana entre cada atomo de carbon alfalfa
    for v in coord:
        distancia_un_atomo = []
        for av in coord:
            distancia = pdist(np.array([v, av]), metric='euclidean').item()
            distancia_un_atomo.append(distancia)
        distancias.append(distancia_un_atomo)

    # se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=index, columns=index, data=distancias)
    return(df_da, index)


df_distancias1, index1 = get_df_distancias(pdb11)
df_distancias2, index2 = get_df_distancias(pdb22)

In [6]:
# se generan cliques, tte devuleve dataframe con cliques de 3 y la lista de cliques sin partir
df_cliques1, cliques1 = fc.gen_3_cliques(df_distancias1, dth=10, k=num_cliques)
print('**'*50)
df_cliques2, cliques2 = fc.gen_3_cliques(df_distancias2, dth=10, k=num_cliques)
print('**'*50)

numero de cliques maximos encontrados: 158
numero de 3-cliques posibles: 1787
****************************************************************************************************
numero de cliques maximos encontrados: 246
numero de 3-cliques posibles: 2102
****************************************************************************************************


In [7]:
def get_df_ca(list_of_residues):
    """Genera dataframe con la informacion necesaria para las siguientes funciones
    FALTA DOCUMENTAR ESTA COSA!!!!"""
    #crear df_ca
    atom_number = []
    atom_name = []
    residue_name = []
    residue_number = []
    coord = []
    for res in list_of_residues:
        for atom in res.atoms:
            atom_number.append(atom.atom_number)
            atom_name.append(atom.name)
            residue_name.append(res.resn)
            residue_number.append(res.resi)
            coord.append(atom.coord)

    df_atoms = pd.DataFrame(columns=['atom_number', 'atom_name', 'residue_name',
                                   'residue_number', 'vector'])
    df_atoms.atom_number = atom_number
    df_atoms.atom_name = atom_name
    df_atoms.residue_name = residue_name
    df_atoms.residue_number = residue_number
    df_atoms.vector = coord

    return(df_atoms)


# CREAR DF_atomos_CA #
df_atoms1 = get_df_ca(pdb11)
df_atoms2 = get_df_ca(pdb22)

In [8]:
# se le pega la estructura secundaria al dataframe de los cliques
# esto va a cambiar por que lo tiene que obtener del objeto residuo
# ya se crea en ss1 y no cuesta reevaluar si es mejor desde el residuo
# checar que es mas rapido si desde residuo o desde dataframe ss
df_cliques1 = fc.paste_SS(ss1, df_cliques1, num_cliques = num_cliques)
df_cliques2 = fc.paste_SS(ss2, df_cliques2, num_cliques = num_cliques)

In [9]:
# comparacion SSM #aqui se obtienen los candidatos posibles pasando el filtro de SS
candidatos_ss = fc.compare_SS(df_cliques1,df_cliques2, num_cliques=num_cliques)

In [10]:
# get coords of cliques
df_cliques1 = fc.get_coords_clique(df_atoms1, df_cliques1, num_cliques)
df_cliques2 = fc.get_coords_clique(df_atoms2, df_cliques2, num_cliques)

In [11]:
# baricentro clique
df_cliques1 = fc.baricenter_clique(df_cliques1, num_cliques)
df_cliques2 = fc.baricenter_clique(df_cliques2, num_cliques)

In [12]:
# vectores gorro
df_cliques1 = fc.center_vectors(df_cliques1, num_cliques)
df_cliques2 = fc.center_vectors(df_cliques2, num_cliques)

In [13]:
for i,j in enumerate(df_cliques1.columns):
    print(i,j)

0 0
1 1
2 2
3 ss_0
4 ss_1
5 ss_2
6 coord_clique_0
7 coord_clique_1
8 coord_clique_2
9 matriz_coordenadas
10 baricentro_clique
11 vec_gorro_0
12 vec_gorro_1
13 vec_gorro_2
14 vectores_gorro


In [14]:
idx_rmsd1, idx_rmsd2 = 3*num_cliques, 4*num_cliques+3
# print(list(range(idx_rmsd1,idx_rmsd2)))
# se pasan a numpy arrays para mayor rapidez
array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)] #del 9 al 15
array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

In [15]:
#calculo del RMSD
print(len(candidatos_ss))

1705808


# Codigo que restringe candidatos

In [16]:
df_cliques1 = fc.get_distancia_promedio(num_cliques, df_cliques1)
df_cliques2 = fc.get_distancia_promedio(num_cliques, df_cliques2)

array_dist_promedio1 = df_cliques1.values[:, -1]  # el ultimo valor de distancia.
array_dist_promedio2 = df_cliques2.values[:, -1]

In [17]:
limite_distancia_minima = 0.45
if num_cliques == 4:
    limite_distancia_minima = 0.9
if num_cliques == 5:
    limite_distancia_minima = 1.8
if num_cliques == 6:
    limite_distancia_minima = 3.6
if num_cliques == 7:
    limite_distancia_minima = 4.5
if num_cliques == 8:
    limite_distancia_minima = 8.0

candidatos_filter_dist = [(i, j) for i, j in candidatos_ss if (
        array_dist_promedio1[i] - array_dist_promedio2[j] >= -limite_distancia_minima) & (
        array_dist_promedio1[i] - array_dist_promedio2[j] <= limite_distancia_minima)]

print(len(candidatos_ss))
print(len(candidatos_filter_dist))

1705808
253993


In [18]:
restriccion_rmsd = 0.15
if num_cliques == 4:
    restriccion_rmsd = 0.30
if num_cliques == 5:
    restriccion_rmsd = 0.60
if num_cliques == 6:
    restriccion_rmsd = 0.90
if num_cliques == 7:
    restriccion_rmsd = 1.50
if num_cliques == 8:
    restriccion_rmsd = 1.80

# Haciendo MUltiprocessing

In [19]:
p = multiprocessing.Pool(multiprocessing.cpu_count()-1)

In [20]:
%%time
rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,  
                    array_cliques1 = array_df_cliques1, 
                    array_cliques2 = array_df_cliques2, 
                    num_cliques = num_cliques), candidatos_filter_dist)

p.close()
p.join()

CPU times: user 7.03 s, sys: 93.2 ms, total: 7.12 s
Wall time: 13.8 s


In [21]:
len(rmsd_1)

253993

In [22]:
f1 = pd.DataFrame(rmsd_1)
f1.columns = ['rmsd','candidatos']
f1['cumple'] = np.where(f1.rmsd <= restriccion_rmsd,1,0)
f1['candidato_clique_1'] = f1.candidatos.str.get(0)
f1['candidato_clique_2'] = f1.candidatos.str.get(1)
f1.shape

(253993, 5)

In [23]:
print(f1[f1.cumple==1].cumple.sum())
f1 = f1[f1.cumple==1]

6907


# Probando la generacion de una nueva tabla con cliques_4 si se empieza con cliques_3

In [24]:
candidatos_df = f1
candidatos_df.sort_values(['candidato_clique_1','candidato_clique_2']).head()

,rmsd,candidatos,cumple,candidato_clique_1,candidato_clique_2
123,0.14247,"(10, 616)",1,10,616
166,0.14225,"(10, 1085)",1,10,1085
273,0.14233,"(10, 1403)",1,10,1403
291,0.14044,"(10, 1456)",1,10,1456
308,0.11607,"(10, 1664)",1,10,1664


In [25]:
def get_cliques_4(df_cliques,col,cliques):
    set_clique = [df_cliques.iloc[i, [0,1,2]].values for i in candidatos_df[col].unique()] #conjunto de candidatos unicos
    lista_chidi = [] # aqui se guardara la lista de numero de residuo
    for muestra in set_clique: # este va a cambiar cada iteracion
        for i in cliques:
            if set(muestra).issubset(i):       # si esta en un clique maximal
                no_estan_en_clique = set(i).difference(set(muestra)) #obten los elementos que no estan en ese clique que si estan en el clique maximal
                for j in no_estan_en_clique: 
                    muestra_falsa = muestra.copy() #se genera una copia para no borrar el orginal
                    muestra_falsa = np.append(muestra_falsa,j) #se apila un elemento de los que no estan
                    if set(muestra_falsa) not in lista_chidi:
                        lista_chidi.append(set(muestra_falsa)) #si no esta en la lista pasa

    return(pd.DataFrame(lista_chidi))

In [26]:
new_df_cliques1 = get_cliques_4(df_cliques1, 'candidato_clique_1',cliques1)
new_df_cliques2 = get_cliques_4(df_cliques2, 'candidato_clique_2',cliques2)
num_cliques = num_cliques + 1

# VA DE NUEZ EL CALCULO

In [27]:
# se le pega la estructura secundaria al dataframe de los cliques
# esto va a cambiar por que lo tiene que obtener del objeto residuo
# ya se crea en ss1 y no cuesta reevaluar si es mejor desde el residuo
# checar que es mas rapido si desde residuo o desde dataframe ss
new_df_cliques1 = fc.paste_SS(ss1, new_df_cliques1, num_cliques = num_cliques)
new_df_cliques2 = fc.paste_SS(ss2, new_df_cliques2, num_cliques = num_cliques)

In [28]:
# comparacion SSM #aqui se obtienen los candidatos posibles pasando el filtro de SS
candidatos_ss = fc.compare_SS(new_df_cliques1,new_df_cliques2, num_cliques=num_cliques)

In [29]:
# get coords of cliques
df_cliques1 = fc.get_coords_clique(df_atoms1, new_df_cliques1, num_cliques)
df_cliques2 = fc.get_coords_clique(df_atoms2, new_df_cliques2, num_cliques)

In [30]:
# baricentro clique
df_cliques1 = fc.baricenter_clique(df_cliques1, num_cliques)
df_cliques2 = fc.baricenter_clique(df_cliques2, num_cliques)

In [31]:
# vectores gorro
df_cliques1 = fc.center_vectors(df_cliques1, num_cliques)
df_cliques2 = fc.center_vectors(df_cliques2, num_cliques)

In [32]:
idx_rmsd1, idx_rmsd2 = 3*num_cliques, 4*num_cliques+3
# print(list(range(idx_rmsd1,idx_rmsd2)))
# se pasan a numpy arrays para mayor rapidez
array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)] #del 9 al 15
array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

In [33]:
#calculo del RMSD
print(len(candidatos_ss))

2434300


# Codigo que restringe candidatos

In [34]:
df_cliques1 = fc.get_distancia_promedio(num_cliques, df_cliques1)
df_cliques2 = fc.get_distancia_promedio(num_cliques, df_cliques2)

array_dist_promedio1 = df_cliques1.values[:, -1]  # el ultimo valor de distancia.
array_dist_promedio2 = df_cliques2.values[:, -1]

In [35]:
limite_distancia_minima = 0.45
if num_cliques == 4:
    limite_distancia_minima = 0.9
if num_cliques == 5:
    limite_distancia_minima = 1.8
if num_cliques == 6:
    limite_distancia_minima = 3.6
if num_cliques == 7:
    limite_distancia_minima = 4.5
if num_cliques == 8:
    limite_distancia_minima = 8.0

In [36]:
#filtro por distancia minima
candidatos_filter_dist = [(i, j) for i, j in candidatos_ss if (
        array_dist_promedio1[i] - array_dist_promedio2[j] >= -limite_distancia_minima) & (
        array_dist_promedio1[i] - array_dist_promedio2[j] <= limite_distancia_minima)]

print('num candidatos filtro SS', len(candidatos_ss))
print('num candidatos filtro distancia y ss', len(candidatos_filter_dist))

num candidatos filtro SS 2434300
num candidatos filtro distancia y ss 869504


# Haciendo MUltiprocessing

In [38]:
p = multiprocessing.Pool(multiprocessing.cpu_count()-1)
restriccion_rmsd = 0.15
if num_cliques == 4:
    restriccion_rmsd = 0.30
if num_cliques == 5:
    restriccion_rmsd = 0.60
if num_cliques == 6:
    restriccion_rmsd = 0.90
if num_cliques == 7:
    restriccion_rmsd = 1.50
if num_cliques == 8:
    restriccion_rmsd = 1.80

In [39]:
%%time
rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,  
                    array_cliques1 = array_df_cliques1, 
                    array_cliques2 = array_df_cliques2, 
                    num_cliques = num_cliques), candidatos_filter_dist)

p.close()
p.join()

CPU times: user 13.6 s, sys: 562 ms, total: 14.2 s
Wall time: 51.4 s


In [40]:
len(rmsd_1)

869504

In [41]:
f1 = pd.DataFrame(rmsd_1)
f1.columns = ['rmsd','candidatos']
f1['cumple'] = np.where(f1.rmsd <= restriccion_rmsd,1,0)
f1['candidato_clique_1'] = f1.candidatos.str.get(0)
f1['candidato_clique_2'] = f1.candidatos.str.get(1)
f1.shape

(869504, 5)

In [42]:
print(f1[f1.cumple==1].cumple.sum())
f1 = f1[f1.cumple==1]

3578


# Probando la generacion de una nueva tabla con cliques_5 si se empieza con cliques_4

In [43]:
candidatos_df = f1
candidatos_df.sort_values(['candidato_clique_1','candidato_clique_2']).head()

,rmsd,candidatos,cumple,candidato_clique_1,candidato_clique_2
7,0.29109,"(0, 10)",1,0,10
42,0.29299,"(0, 57)",1,0,57
416,0.27244,"(0, 1035)",1,0,1035
664,0.29610,"(1, 86)",1,1,86
736,0.28929,"(1, 191)",1,1,191


In [44]:
def get_cliques_5(df_cliques,col,cliques):
    set_clique = [df_cliques.iloc[i, [0,1,2,3]].values for i in candidatos_df[col].unique()] #conjunto de candidatos unicos
    lista_chidi = [] # aqui se guardara la lista de numero de residuo
    for muestra in set_clique: # este va a cambiar cada iteracion
        for i in cliques:
            if set(muestra).issubset(i):       # si esta en un clique maximal
                no_estan_en_clique = set(i).difference(set(muestra)) #obten los elementos que no estan en ese clique que si estan en el clique maximal
                for j in no_estan_en_clique: 
                    muestra_falsa = muestra.copy() #se genera una copia para no borrar el orginal
                    muestra_falsa = np.append(muestra_falsa,j) #se apila un elemento de los que no estan
                    if set(muestra_falsa) not in lista_chidi:
                        lista_chidi.append(set(muestra_falsa)) #si no esta en la lista pasa

    return(pd.DataFrame(lista_chidi))

In [45]:
new_df_cliques1 = get_cliques_5(df_cliques1, 'candidato_clique_1',cliques1)
new_df_cliques2 = get_cliques_5(df_cliques2, 'candidato_clique_2',cliques2)
num_cliques = num_cliques + 1

# VA DE NUEZ EL CALCULO

In [46]:
# se le pega la estructura secundaria al dataframe de los cliques
# esto va a cambiar por que lo tiene que obtener del objeto residuo
# ya se crea en ss1 y no cuesta reevaluar si es mejor desde el residuo
# checar que es mas rapido si desde residuo o desde dataframe ss
new_df_cliques1 = fc.paste_SS(ss1, new_df_cliques1, num_cliques = num_cliques)
new_df_cliques2 = fc.paste_SS(ss2, new_df_cliques2, num_cliques = num_cliques)

In [47]:
# comparacion SSM #aqui se obtienen los candidatos posibles pasando el filtro de SS
candidatos_ss = fc.compare_SS(new_df_cliques1,new_df_cliques2, num_cliques=num_cliques)

In [48]:
# get coords of cliques
df_cliques1 = fc.get_coords_clique(df_atoms1, new_df_cliques1, num_cliques)
df_cliques2 = fc.get_coords_clique(df_atoms2, new_df_cliques2, num_cliques)

In [49]:
# baricentro clique
df_cliques1 = fc.baricenter_clique(df_cliques1, num_cliques)
df_cliques2 = fc.baricenter_clique(df_cliques2, num_cliques)

In [50]:
# vectores gorro
df_cliques1 = fc.center_vectors(df_cliques1, num_cliques)
df_cliques2 = fc.center_vectors(df_cliques2, num_cliques)

In [51]:
idx_rmsd1, idx_rmsd2 = 3*num_cliques, 4*num_cliques+3
# print(list(range(idx_rmsd1,idx_rmsd2)))
# se pasan a numpy arrays para mayor rapidez
array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)] #del 9 al 15
array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

In [52]:
#calculo del RMSD
print(len(candidatos_ss))

1185342


# Codigo que restringe candidatos

In [53]:
df_cliques1 = fc.get_distancia_promedio(num_cliques, df_cliques1)
df_cliques2 = fc.get_distancia_promedio(num_cliques, df_cliques2)

array_dist_promedio1 = df_cliques1.values[:, -1]  # el ultimo valor de distancia.
array_dist_promedio2 = df_cliques2.values[:, -1]

In [54]:
limite_distancia_minima = 0.45
if num_cliques == 4:
    limite_distancia_minima = 0.9
if num_cliques == 5:
    limite_distancia_minima = 1.8
if num_cliques == 6:
    limite_distancia_minima = 3.6
if num_cliques == 7:
    limite_distancia_minima = 4.5
if num_cliques == 8:
    limite_distancia_minima = 8.0

In [55]:
#filtro por distancia minima
candidatos_filter_dist = [(i, j) for i, j in candidatos_ss if (
        array_dist_promedio1[i] - array_dist_promedio2[j] >= -limite_distancia_minima) & (
        array_dist_promedio1[i] - array_dist_promedio2[j] <= limite_distancia_minima)]

print('num candidatos filtro SS', len(candidatos_ss))
print('num candidatos filtro distancia y ss', len(candidatos_filter_dist))

num candidatos filtro SS 1185342
num candidatos filtro distancia y ss 900886


# Haciendo MUltiprocessing

In [56]:
p = multiprocessing.Pool(multiprocessing.cpu_count()-1)
restriccion_rmsd = 0.15
if num_cliques == 4:
    restriccion_rmsd = 0.30
if num_cliques == 5:
    restriccion_rmsd = 0.60
if num_cliques == 6:
    restriccion_rmsd = 0.90
if num_cliques == 7:
    restriccion_rmsd = 1.50
if num_cliques == 8:
    restriccion_rmsd = 1.80

In [57]:
%%time
rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,  
                    array_cliques1 = array_df_cliques1, 
                    array_cliques2 = array_df_cliques2, 
                    num_cliques = num_cliques), candidatos_filter_dist)

p.close()
p.join()

CPU times: user 11.9 s, sys: 264 ms, total: 12.1 s
Wall time: 57.2 s


In [58]:
len(rmsd_1)

900886

In [59]:
f1 = pd.DataFrame(rmsd_1)
f1.columns = ['rmsd','candidatos']
f1['cumple'] = np.where(f1.rmsd <= restriccion_rmsd,1,0)
f1['candidato_clique_1'] = f1.candidatos.str.get(0)
f1['candidato_clique_2'] = f1.candidatos.str.get(1)
f1.shape

(900886, 5)

In [60]:
print(f1[f1.cumple==1].cumple.sum())
f1 = f1[f1.cumple==1]

43075


# Probando la generacion de una nueva tabla con cliques_6 si se empieza con cliques_5

In [61]:
candidatos_df = f1
candidatos_df.sort_values(['candidato_clique_1','candidato_clique_2']).head()

,rmsd,candidatos,cumple,candidato_clique_1,candidato_clique_2
0,0.33304,"(0, 0)",1,0,0
1,0.37599,"(0, 1)",1,0,1
3,0.49367,"(0, 3)",1,0,3
13,0.55671,"(0, 14)",1,0,14
18,0.45587,"(0, 19)",1,0,19


In [62]:
def get_cliques_6(df_cliques,col,cliques):
    set_clique = [df_cliques.iloc[i, [0,1,2,3,4]].values for i in candidatos_df[col].unique()] #conjunto de candidatos unicos
    lista_chidi = [] # aqui se guardara la lista de numero de residuo
    for muestra in set_clique: # este va a cambiar cada iteracion
        for i in cliques:
            if set(muestra).issubset(i):       # si esta en un clique maximal
                no_estan_en_clique = set(i).difference(set(muestra)) #obten los elementos que no estan en ese clique que si estan en el clique maximal
                for j in no_estan_en_clique: 
                    muestra_falsa = muestra.copy() #se genera una copia para no borrar el orginal
                    muestra_falsa = np.append(muestra_falsa,j) #se apila un elemento de los que no estan
                    if set(muestra_falsa) not in lista_chidi:
                        lista_chidi.append(set(muestra_falsa)) #si no esta en la lista pasa

    return(pd.DataFrame(lista_chidi))

In [63]:
new_df_cliques1 = get_cliques_6(df_cliques1, 'candidato_clique_1',cliques1)
new_df_cliques2 = get_cliques_6(df_cliques2, 'candidato_clique_2',cliques2)
num_cliques = num_cliques + 1

# VA DE NUEZ EL CALCULO

In [64]:
# se le pega la estructura secundaria al dataframe de los cliques
# esto va a cambiar por que lo tiene que obtener del objeto residuo
# ya se crea en ss1 y no cuesta reevaluar si es mejor desde el residuo
# checar que es mas rapido si desde residuo o desde dataframe ss
new_df_cliques1 = fc.paste_SS(ss1, new_df_cliques1, num_cliques = num_cliques)
new_df_cliques2 = fc.paste_SS(ss2, new_df_cliques2, num_cliques = num_cliques)

In [65]:
# comparacion SSM #aqui se obtienen los candidatos posibles pasando el filtro de SS
candidatos_ss = fc.compare_SS(new_df_cliques1,new_df_cliques2, num_cliques=num_cliques)

In [66]:
# get coords of cliques
df_cliques1 = fc.get_coords_clique(df_atoms1, new_df_cliques1, num_cliques)
df_cliques2 = fc.get_coords_clique(df_atoms2, new_df_cliques2, num_cliques)

In [67]:
# baricentro clique
df_cliques1 = fc.baricenter_clique(df_cliques1, num_cliques)
df_cliques2 = fc.baricenter_clique(df_cliques2, num_cliques)

In [68]:
# vectores gorro
df_cliques1 = fc.center_vectors(df_cliques1, num_cliques)
df_cliques2 = fc.center_vectors(df_cliques2, num_cliques)

In [69]:
idx_rmsd1, idx_rmsd2 = 3*num_cliques, 4*num_cliques+3
# print(list(range(idx_rmsd1,idx_rmsd2)))
# se pasan a numpy arrays para mayor rapidez
array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)] #del 9 al 15
array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

In [70]:
#calculo del RMSD
print(len(candidatos_ss))

337448


# Codigo que restringe candidatos

In [71]:
df_cliques1 = fc.get_distancia_promedio(num_cliques, df_cliques1)
df_cliques2 = fc.get_distancia_promedio(num_cliques, df_cliques2)

array_dist_promedio1 = df_cliques1.values[:, -1]  # el ultimo valor de distancia.
array_dist_promedio2 = df_cliques2.values[:, -1]

In [72]:
limite_distancia_minima = 0.45
if num_cliques == 4:
    limite_distancia_minima = 0.9
if num_cliques == 5:
    limite_distancia_minima = 1.8
if num_cliques == 6:
    limite_distancia_minima = 3.6
if num_cliques == 7:
    limite_distancia_minima = 4.5
if num_cliques == 8:
    limite_distancia_minima = 8.0

In [73]:
#filtro por distancia minima
candidatos_filter_dist = [(i, j) for i, j in candidatos_ss if (
        array_dist_promedio1[i] - array_dist_promedio2[j] >= -limite_distancia_minima) & (
        array_dist_promedio1[i] - array_dist_promedio2[j] <= limite_distancia_minima)]

print('num candidatos filtro SS', len(candidatos_ss))
print('num candidatos filtro distancia y ss', len(candidatos_filter_dist))

num candidatos filtro SS 337448
num candidatos filtro distancia y ss 332695


# Haciendo MUltiprocessing

In [74]:
p = multiprocessing.Pool(multiprocessing.cpu_count()-1)
restriccion_rmsd = 0.15
if num_cliques == 4:
    restriccion_rmsd = 0.30
if num_cliques == 5:
    restriccion_rmsd = 0.60
if num_cliques == 6:
    restriccion_rmsd = 0.90
if num_cliques == 7:
    restriccion_rmsd = 1.50
if num_cliques == 8:
    restriccion_rmsd = 1.80

In [75]:
%%time
rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,  
                    array_cliques1 = array_df_cliques1, 
                    array_cliques2 = array_df_cliques2, 
                    num_cliques = num_cliques), candidatos_filter_dist)

p.close()
p.join()

CPU times: user 8.21 s, sys: 188 ms, total: 8.4 s
Wall time: 24.8 s


In [76]:
len(rmsd_1)

332695

In [77]:
f1 = pd.DataFrame(rmsd_1)
f1.columns = ['rmsd','candidatos']
f1['cumple'] = np.where(f1.rmsd <= restriccion_rmsd,1,0)
f1['candidato_clique_1'] = f1.candidatos.str.get(0)
f1['candidato_clique_2'] = f1.candidatos.str.get(1)
f1.shape

(332695, 5)

In [78]:
print(f1[f1.cumple==1].cumple.sum())
f1 = f1[f1.cumple==1]

26876


# Probando la generacion de una nueva tabla con cliques_7 si se empieza con cliques_6

In [80]:
candidatos_df = f1
candidatos_df.sort_values(['candidato_clique_1','candidato_clique_2']).head()

,rmsd,candidatos,cumple,candidato_clique_1,candidato_clique_2
1,0.88576,"(0, 1)",1,0,1
3,0.88959,"(0, 3)",1,0,3
30,0.78204,"(0, 32)",1,0,32
73,0.85510,"(0, 102)",1,0,102
434,0.46614,"(2, 0)",1,2,0


In [81]:
def get_cliques_7(df_cliques,col,cliques):
    set_clique = [df_cliques.iloc[i, [0,1,2,3,4,5]].values for i in candidatos_df[col].unique()] #conjunto de candidatos unicos
    lista_chidi = [] # aqui se guardara la lista de numero de residuo
    for muestra in set_clique: # este va a cambiar cada iteracion
        for i in cliques:
            if set(muestra).issubset(i):       # si esta en un clique maximal
                no_estan_en_clique = set(i).difference(set(muestra)) #obten los elementos que no estan en ese clique que si estan en el clique maximal
                for j in no_estan_en_clique: 
                    muestra_falsa = muestra.copy() #se genera una copia para no borrar el orginal
                    muestra_falsa = np.append(muestra_falsa,j) #se apila un elemento de los que no estan
                    if set(muestra_falsa) not in lista_chidi:
                        lista_chidi.append(set(muestra_falsa)) #si no esta en la lista pasa

    return(pd.DataFrame(lista_chidi))

In [82]:
new_df_cliques1 = get_cliques_7(df_cliques1, 'candidato_clique_1',cliques1)
new_df_cliques2 = get_cliques_7(df_cliques2, 'candidato_clique_2',cliques2)
num_cliques = num_cliques + 1

# VA DE NUEZ EL CALCULO

In [83]:
# se le pega la estructura secundaria al dataframe de los cliques
# esto va a cambiar por que lo tiene que obtener del objeto residuo
# ya se crea en ss1 y no cuesta reevaluar si es mejor desde el residuo
# checar que es mas rapido si desde residuo o desde dataframe ss
new_df_cliques1 = fc.paste_SS(ss1, new_df_cliques1, num_cliques = num_cliques)
new_df_cliques2 = fc.paste_SS(ss2, new_df_cliques2, num_cliques = num_cliques)

In [84]:
# comparacion SSM #aqui se obtienen los candidatos posibles pasando el filtro de SS
candidatos_ss = fc.compare_SS(new_df_cliques1,new_df_cliques2, num_cliques=num_cliques)

In [85]:
# get coords of cliques
df_cliques1 = fc.get_coords_clique(df_atoms1, new_df_cliques1, num_cliques)
df_cliques2 = fc.get_coords_clique(df_atoms2, new_df_cliques2, num_cliques)

In [86]:
# baricentro clique
df_cliques1 = fc.baricenter_clique(df_cliques1, num_cliques)
df_cliques2 = fc.baricenter_clique(df_cliques2, num_cliques)

In [87]:
# vectores gorro
df_cliques1 = fc.center_vectors(df_cliques1, num_cliques)
df_cliques2 = fc.center_vectors(df_cliques2, num_cliques)

In [88]:
idx_rmsd1, idx_rmsd2 = 3*num_cliques, 4*num_cliques+3
# print(list(range(idx_rmsd1,idx_rmsd2)))
# se pasan a numpy arrays para mayor rapidez
array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)] #del 9 al 15
array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

In [89]:
#calculo del RMSD
print(len(candidatos_ss))

28023


# Codigo que restringe candidatos

In [90]:
df_cliques1 = fc.get_distancia_promedio(num_cliques, df_cliques1)
df_cliques2 = fc.get_distancia_promedio(num_cliques, df_cliques2)

array_dist_promedio1 = df_cliques1.values[:, -1]  # el ultimo valor de distancia.
array_dist_promedio2 = df_cliques2.values[:, -1]

In [91]:
limite_distancia_minima = 0.45
if num_cliques == 4:
    limite_distancia_minima = 0.9
if num_cliques == 5:
    limite_distancia_minima = 1.8
if num_cliques == 6:
    limite_distancia_minima = 3.6
if num_cliques == 7:
    limite_distancia_minima = 4.5
if num_cliques == 8:
    limite_distancia_minima = 8.0

In [92]:
#filtro por distancia minima
candidatos_filter_dist = [(i, j) for i, j in candidatos_ss if (
        array_dist_promedio1[i] - array_dist_promedio2[j] >= -limite_distancia_minima) & (
        array_dist_promedio1[i] - array_dist_promedio2[j] <= limite_distancia_minima)]

print('num candidatos filtro SS', len(candidatos_ss))
print('num candidatos filtro distancia y ss', len(candidatos_filter_dist))

num candidatos filtro SS 28023
num candidatos filtro distancia y ss 28010


# Haciendo MUltiprocessing

In [93]:
p = multiprocessing.Pool(multiprocessing.cpu_count()-1)
restriccion_rmsd = 0.15
if num_cliques == 4:
    restriccion_rmsd = 0.30
if num_cliques == 5:
    restriccion_rmsd = 0.60
if num_cliques == 6:
    restriccion_rmsd = 0.90
if num_cliques == 7:
    restriccion_rmsd = 1.50
if num_cliques == 8:
    restriccion_rmsd = 1.80

In [94]:
%%time
rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,  
                    array_cliques1 = array_df_cliques1, 
                    array_cliques2 = array_df_cliques2, 
                    num_cliques = num_cliques), candidatos_filter_dist)

p.close()
p.join()

CPU times: user 2.37 s, sys: 55.9 ms, total: 2.43 s
Wall time: 2.97 s


In [95]:
len(rmsd_1)

28010

In [96]:
f1 = pd.DataFrame(rmsd_1)
f1.columns = ['rmsd','candidatos']
f1['cumple'] = np.where(f1.rmsd <= restriccion_rmsd,1,0)
f1['candidato_clique_1'] = f1.candidatos.str.get(0)
f1['candidato_clique_2'] = f1.candidatos.str.get(1)
f1.shape

(28010, 5)

In [97]:
print(f1[f1.cumple==1].cumple.sum())
f1 = f1[f1.cumple==1]

12114


In [99]:
ahora = datetime.datetime.now()

In [101]:
print(ahora - tiempo)

0:04:32.441614
